# Utils

In [99]:
import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt


imgCol = cv2.imread('../Utils/LenaJPEG.jpg')
imgCol.shape

def plotSignal(signal=[], title='', xmin = 0, xmax = 0):
  if(xmax == 0):
    xmax = signal.size

  plt.title(title)
  plt.plot(signal)
  plt.xlim(xmin, xmax)
  plt.show() 

def show_Image(img):
    cv2.imshow("img", img)
    cv2.waitKey(0)
    cv2.destroyAll()

# Praktikum 01

In [1]:
def printNumberFrom1To100():
    for i in range(1, 101):
        print(i)

def fibunacci(n):
    a = [0]*(n+4)
    a[0] = 1
    a[1] = 1
    print(0)
    print(1)
    print(1)
    for i in range(2,n+3):
        a[i] = a[i-1]+a[i-2]
        print(a[i])

# fibunacci(10)

# for y in range(height):
#     for x in range(width):
#         imgCol[y][x] = pow(imgCol[y][x]/255, 1/2) * 255



# cv2.imshow("img", imgCol)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


# Praktikum 02

In [100]:
def getHistogramm(img=[]):
    histogram=np.zeros(256)
    height, width, depth = img.shape
    for y in range(height):
        for x in range(width):
            if img.ndim == 3:
                i = 0.2125 * img[y][x][0] + 0.7154 * img[y][x][1] + 0.072 * img[y][x][2]
            else:
                i = img[y][x]

            histogram[int(i)] += 1

    return histogram

def getNormHistogramm(img=[]):
    height, width, depth = img.shape
    return getHistogramm(img) / (width*height)

def getCumulatedHistogramm(img):
    H = np.zeros(256)
    h = getNormHistogramm(img)
    for i in range(len(h)):
        for j in range(i): 
            H[i] += h[j]
    return H

def linearContrast(img=[], t0=0, t1=255):
    height, width, depth = img.shape
    toRet = np.zeros((height, width), dtype='uint8')
    for y in range(height):
        for x in range(width):
            a = float(0.2125 * img[y][x][0] + 0.7154 * img[y][x][1] + 0.072 * img[y][x][2])
            if a < t0:
                toRet[y][x] = 0
            if t0 <= a < t1:
                toRet[y][x] = 255/(t1-t0) * (a-t0)
            if a >= t1:
                toRet[y][x] = 255
    return toRet

def autoContrast(img, schwellwert):
    a1_low = getPercentage(img, schwellwert)
    a1_high = getPercentage(img, 1-schwellwert)
    return linearContrast(img, a1_low, a1_high)

def getPercentage(img, schwellwert):
    H = getCumulatedHistogramm(img)
    toRet = 255
    for i in range(len(H)):
        if H[i] >= schwellwert:
            toRet = i
            break
    return toRet

def getMedianOfHistogramm(img):
    H = getCumulatedHistogramm(img)
    for i in range(len(H)):
        if H[i] >= 0.5:
            return i


def getBinary(img=[], schwellwert=125):
    height, width, depth = img.shape
    toRet = np.zeros((height, width))
    for y in range(height):
        for x in range(width):
            a = 0.2125 * img[y][x][0] + 0.7154 * img[y][x][1] + 0.072 * img[y][x][2]
            if a < schwellwert:
                toRet[y][x] = 0
            else:
                toRet[y][x] = 1
    return toRet

def same_image(img=[]):
    height, width, depth = img.shape
    toRet = np.zeros((width, height), dtype='uint8')
    for y in range(height):
        for x in range(width):
            a = 0.2125 * img[y][x][0] + 0.7154 * img[y][x][1] + 0.072 * img[y][x][2]
            # a = img[y][x][0]
            toRet[y][x] = a
            
    return toRet

testing Praktikum 02

In [101]:
# print(getMedianOfHistogramm(imgCol))
median = getMedianOfHistogramm(imgCol)
print(median)
img = getBinary(imgCol, median)
# auto_con = autoContrast(imgCol, 0.1)

# plotSignal(getCumulatedHistogramm(imgCol))
# show_Image(img)
# show_Image(imgCol)

128


# Praktikum 03

In [208]:
def Otsu_Threshold_naiv(Histrogramm=[], img=[]):
    toRet = -1
    varMax = -1
    width, height, depth = img.shape
    size = width * height
    K = len(Histrogramm)
    for i in range(K-1):
        w_0 = np.sum(Histrogramm[0:i]) / size
        w_1 = np.sum(Histrogramm[i:K]) / size
        my_0 = np.mean(Histrogramm[0:i])
        my_1 = np.mean(Histrogramm[i:K])
        varBetween = w_0 * w_1 * (my_0-my_1)**2
        if varMax < varBetween:
            varMax = varBetween
            toRet = i
    return toRet

def weigthed_Histogram(h=[]):
    toRet = 0
    for i in range(len(h)):
        toRet += i * h[i]
    return toRet


def Otsu_Threshold_efficient(h=[], img=[]):
    toRet = -1
    varMax = -1
    c_0 = 0
    sum_0 = 0 
    width, height, depth = img.shape
    size = width * height
    K = len(h)
    for i in range(K):
        c_0 = c_0 + h[i]
        c_1 = size - c_0
        sum_0 = sum_0 + i * h[i]
        my_0 = sum_0 / c_0
        my_1 = ( weigthed_Histogram(h) - sum_0) / c_1
        varBetween = c_0 * c_1 * (my_0-my_1)**2
        if varMax < varBetween:
            varMax = varBetween
            toRet = i
    return toRet

def getOpen(img=[], schwellwert=125):
    height, width, depth = img.shape
    toRet = np.zeros((height, width))
    for y in range(height):
        for x in range(width):
            a = 0.2125 * img[y][x][0] + 0.7154 * img[y][x][1] + 0.072 * img[y][x][2]
            if a < schwellwert:
                toRet[y][x] = 0
            else:
                toRet[y][x] = 1
        kernel = np.array([[0, 1, 0],
                           [1, 1, 1],
                           [0, 1, 0]], dtype=np.uint8)
        
    # toRet = cv2.erode(toRet, kernel)
    # toRet = cv2.erode(toRet, kernel)
    # toRet = cv2.dilate(toRet, kernel)
    return toRet

def region_labeling(img=[]):
    toRet = img
    height, width = img.shape
    m = 2
    for y in range(height):
        for x in range(width):
            if toRet[y][x] == 1:
                floodfill(toRet, x, y, m)
                m+=1
    return toRet

def floodfill(img=[], x=1, y=1, m=1):
    height, width = img.shape
    S = []
    S.append((x, y))
    
    while len(S) > 0:
        (x, y) = S.pop()
        if((0<=x<=width and 0<=y<=height) and img[y][x] == 1):
            img[y][x] = m
            S.append((x+1, y))
            S.append((x, y+1))
            S.append((x, y-1))
            S.append((x-1, y))


testing Praktikum 03

In [192]:
SetExmpl = cv2.imread('../Utils/SetWellen.png')
width_set, height_set, depht_set = SetExmpl.shape
setExample_01 = cv2.resize(SetExmpl, [height_set//3, width_set//3])

otsu_naiv = Otsu_Threshold_naiv(getHistogramm(setExample_01), setExample_01)
otsu_efficient = Otsu_Threshold_efficient(getHistogramm(setExample_01), setExample_01)
print(otsu_naiv)
print(otsu_efficient)
median = getMedianOfHistogramm(setExample_01)
print(median)

d:\Stuff\GitHub\Bildverarbeitung1_2023\env\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\Stuff\GitHub\Bildverarbeitung1_2023\env\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\Johan Laptop\AppData\Local\Temp\ipykernel_17132\332052374.py:38: RuntimeWarning: invalid value encountered in scalar divide
  my_1 = ( weigthed_Histogram(h) - sum_0) / c_1


35
115
33


In [207]:

binary = getOpen(setExample_01, 115)
label = region_labeling(binary)
# test = getHistogramm(setExample_01)
show_Image(label)
# show_Image(setExample_01)

AttributeError: module 'cv2' has no attribute 'destroyAll'